In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns


import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis

from gensim.models import word2vec

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

In [4]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = df['op_datetime']
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)
df['op_second2'] = df.groupby('device_num_transform')['op_second'].shift(2)
df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())
df['op_diff_second2'] = (df['op_second'] - df['op_second2']).map(lambda x: x.total_seconds())

df['op_diff_second1_tmp'] = df['op_diff_second1']>10
# 客户第几次登录
df['op_times_groups'] = df.groupby('device_num_transform')['op_diff_second1_tmp'].apply(lambda x: x.cumsum())

# 系统层面的一段时间
df['system_op_second'] = df['op_datetime']
df['system_op_second1'] = df['system_op_second'].shift(1)
df['system_op_second2'] = df['system_op_second'].shift(2)
df['system_op_diff_second1'] = (df['system_op_second'] - df['system_op_second1']).map(lambda x: x.total_seconds())
df['system_op_diff_second2'] = (df['system_op_second'] - df['system_op_second2']).map(lambda x: x.total_seconds())

df['system_op_diff_second1_tmp'] = df['system_op_diff_second1']>400
# 客户第几次登录
df['system_op_times_groups'] = df['system_op_diff_second1_tmp'].cumsum()

df = df.drop(columns=['op_second','op_second1','op_diff_second1_tmp', 'op_second2', 
                     'system_op_second2',
                      'system_op_second', 'system_op_second1', 'system_op_diff_second1_tmp'])

In [5]:
cate_cols = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

df = get_time_base(df, cols='op_datetime')
df = get_sequence_statis(df, col='system_op_diff_second1', n=5, freq=3 )
df = get_sequence_groupby_statis(df, col='system_op_diff_second1',cate_cols= cate_cols, n=5, freq=3)
#df = get_sequence_statis(df, col='system_op_diff_second2', n=5, freq=3 )
#df = get_sequence_groupby_statis(df, col='system_op_diff_second2',cate_cols= cate_cols, n=5, freq=3)
df.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,op_days,op_diff_second1,op_diff_second2,op_times_groups,system_op_diff_second1,system_op_diff_second2,system_op_times_groups,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,...,kurt_log_system_transform_system_op_diff_second1_9,avg_log_system_transform_system_op_diff_second1_12,median_log_system_transform_system_op_diff_second1_12,max_log_system_transform_system_op_diff_second1_12,min_log_system_transform_system_op_diff_second1_12,std_log_system_transform_system_op_diff_second1_12,skew_log_system_transform_system_op_diff_second1_12,kurt_log_system_transform_system_op_diff_second1_12,avg_log_system_transform_system_op_diff_second1_15,median_log_system_transform_system_op_diff_second1_15,max_log_system_transform_system_op_diff_second1_15,min_log_system_transform_system_op_diff_second1_15,std_log_system_transform_system_op_diff_second1_15,skew_log_system_transform_system_op_diff_second1_15,kurt_log_system_transform_system_op_diff_second1_15,avg_url_system_op_diff_second1_3,median_url_system_op_diff_second1_3,max_url_system_op_diff_second1_3,min_url_system_op_diff_second1_3,std_url_system_op_diff_second1_3,skew_url_system_op_diff_second1_3,kurt_url_system_op_diff_second1_3,avg_url_system_op_diff_second1_6,median_url_system_op_diff_second1_6,max_url_system_op_diff_second1_6,min_url_system_op_diff_second1_6,std_url_system_op_diff_second1_6,skew_url_system_op_diff_second1_6,kurt_url_system_op_diff_second1_6,avg_url_system_op_diff_second1_9,median_url_system_op_diff_second1_9,max_url_system_op_diff_second1_9,min_url_system_op_diff_second1_9,std_url_system_op_diff_second1_9,skew_url_system_op_diff_second1_9,kurt_url_system_op_diff_second1_9,avg_url_system_op_diff_second1_12,median_url_system_op_diff_second1_12,max_url_system_op_diff_second1_12,min_url_system_op_diff_second1_12,std_url_system_op_diff_second1_12,skew_url_system_op_diff_second1_12,kurt_url_system_op_diff_second1_12,avg_url_system_op_diff_second1_15,median_url_system_op_diff_second1_15,max_url_system_op_diff_second1_15,min_url_system_op_diff_second1_15,std_url_system_op_diff_second1_15,skew_url_system_op_diff_second1_15,kurt_url_system_op_diff_second1_15
44477,44477,xiongkai3397,rd,6H1iPLgBB,GCgxrFb69up7,chrome_93,chrome,win,win10,2022-01-07 02:44:29,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,2022-01-07,NaN,NaN,0,NaN,NaN,0,2022,1,7,2,44,29,1,4,False,False,False,0.104528,-0.994522,-0.994522,-0.104528,0.500000,0.866025,0.988468,0.151428,-0.433884,-0.900969,0.5,0.866025,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45489,45489,zhengguiying7117,rd,0mjaEf4SB,8ftsXFm5I1Ej,safari_13,safari,macos,macos_big_sur_11,2022-01-07 02:54:32,内网,200,成都,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,2022-01-07,NaN,NaN,0,603.0,NaN,1,2022,1,7,2,54,32,1,4,False,False,False,-0.207912,-0.978148,-0.587785,0.809017,0.500000,0.866025,0.988468,0.151428,-0.433884,-0.900969,0.5,0.866025,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45706,45706,yuanjun5870,

In [6]:

time_feats = ['system_op_times_groups', 'op_days', 'op_month']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    i_tmp = df.groupby([i])
    # 系统往前看，处理了多少事
    df['system_{}_cumsum'.format(i)] = i_tmp['helper'].cumsum()
    # 系统往前看，处理了多少坏事
    df['system_{}_error_code_cumsum'.format(i)] = i_tmp['http_status_code_helper'].cumsum()
    
    for j in cate_feats:
        index_set = set(df.groupby([i, j],as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        j_tmp = df.groupby([i, j])
        # 系统往前看，不同维度的处理了多少情况
        df['system_{}_{}_cumunique'.format(i, j)] = j_tmp['tmp_helper'].cumsum()
    

        if j not in ['ip_transform', 'user_name', 'device_num_transform']:
            for k in df[j].unique():
                tmp = df[df[j]==k].groupby([i])

                # 系统往前看，不同维度不同情况分别处理了多少次
                df['system_{}_{}_{}_cumsum'.format(i,j,k)] = tmp['helper'].cumsum()
                # 系统往前看，不同维度不同情况error_code分别处理了多少次
                df['system_{}_{}_{}_error_code_cumsum'.format(i,j,k)] = tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

In [7]:

time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()


for i in time_feats:
    for j in object_feats:
        j_tmp = df.groupby([i,j])
        df['{}_{}_cumsum'.format(i,j)] = j_tmp['helper'].cumsum()
        df['{}_{}_error_code_cumsum'.format(i,j)] = j_tmp['http_status_code_helper'].cumsum()
        
        for k in cate_feats:
            if k == j: continue
            index_set = set(df.groupby([i,j,k], as_index=False).first()['sampler_index_helper'].values)            
            df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
            k_tmp = df.groupby([i,j,k])
            df['{}_{}_{}_cumunique'.format(i,j,k)] = k_tmp['tmp_helper'].cumsum()

            if k not in ['ip_transform', 'user_name', 'device_num_transform']:
                for v in df[k].unique():
                    v_tmp = df[df[k]==v].groupby([i,j])
                    df['{}_{}_{}_{}_cumsum'.format(i,j,k,v)] = v_tmp['helper'].cumsum()
                    df['{}_{}_{}_{}_error_code_cumsum'.format(i,j,k,v)] = v_tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

In [8]:
path_embeding = os.path.join(path_new_data, 'corpus_txt')

res_cols = []
for col in cate_feats:
    df_tmp = pd.read_csv(os.path.join(path_embeding, 'models_{}_embedding.txt'.format(col)), skiprows=1, header=None, sep=' ')
    df_tmp.columns = ['{}_{}'.format(col, i) for i in df_tmp.columns]
    df = pd.merge(left=df, right=df_tmp, how='left', left_on=col, right_on='{}_0'.format(col))
    res_cols.append('{}_0'.format(col))
    

remove_cols = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'op_datetime', 'op_month'] + res_cols

df = df.drop(columns=remove_cols)

In [9]:
df_train = df[df[y_label].notna()].reset_index(drop=True)
df_test = df[df[y_label].isna()].reset_index(drop=True)

In [10]:
feats1 = ['op_datetime_hour',
 'median_user_name_system_op_diff_second1_15',
 'min_department_system_op_diff_second1_15',
 'min_log_system_transform_system_op_diff_second1_3',
 'skew_user_name_system_op_diff_second1_3',
 'median_op_city_system_op_diff_second1_6',
 'max_device_num_transform_system_op_diff_second1_9',
 'median_op_city_system_op_diff_second1_15',
 'skew_ip_transform_system_op_diff_second1_3',
 'min_op_city_system_op_diff_second1_15',
 'max_user_name_system_op_diff_second1_3',
 'median_browser_system_op_diff_second1_15',
 'min_device_num_transform_system_op_diff_second1_6',
 'max_browser_version_system_op_diff_second1_9',
 'median_ip_transform_system_op_diff_second1_12',
 'skew_log_system_transform_system_op_diff_second1_9',
 'min_url_system_op_diff_second1_9',
 'skew_os_type_system_op_diff_second1_6',
 'skew_log_system_transform_system_op_diff_second1_3',
 'min_url_system_op_diff_second1_15',
 'median_device_num_transform_system_op_diff_second1_6',
 'skew_ip_type_system_op_diff_second1_6',
 'min_user_name_system_op_diff_second1_6',
 'max_department_system_op_diff_second1_9',
 'kurt_department_system_op_diff_second1_15',
 'min_browser_system_op_diff_second1_15',
 'median_browser_version_system_op_diff_second1_6',
 'skew_http_status_code_system_op_diff_second1_15',
 'min_department_system_op_diff_second1_6',
 'skew_log_system_transform_system_op_diff_second1_15',
 'median_device_num_transform_system_op_diff_second1_3',
 'median_department_system_op_diff_second1_15',
 'kurt_ip_type_system_op_diff_second1_12',
 'max_device_num_transform_system_op_diff_second1_6',
 'avg_user_name_system_op_diff_second1_9',
 'op_diff_second1',
 'kurt_ip_transform_system_op_diff_second1_6',
 'skew_http_status_code_system_op_diff_second1_3',
 'op_datetime_dayofweek_sin',
 'avg_ip_transform_system_op_diff_second1_6',
 'std_browser_version_system_op_diff_second1_6',
 'kurt_http_status_code_system_op_diff_second1_12',
 'max_log_system_transform_system_op_diff_second1_15',
 'skew_url_system_op_diff_second1_6',
 'kurt_http_status_code_system_op_diff_second1_9',
 'min_op_city_system_op_diff_second1_6',
 'max_ip_transform_system_op_diff_second1_3',
 'skew_department_system_op_diff_second1_9',
 'median_user_name_system_op_diff_second1_9',
 'min_device_num_transform_system_op_diff_second1_15',
 'min_url_system_op_diff_second1_6',
 'op_datetime_hour_cos',
 'min_ip_type_system_op_diff_second1_6',
 'avg_browser_system_op_diff_second1_9',
 'kurt_url_system_op_diff_second1_6',
 'min_http_status_code_system_op_diff_second1_12',
 'avg_ip_type_system_op_diff_second1_3',
 'kurt_http_status_code_system_op_diff_second1_6',
 'min_op_city_system_op_diff_second1_3',
 'kurt_department_system_op_diff_second1_6',
 'avg_os_type_system_op_diff_second1_15',
 'skew_log_system_transform_system_op_diff_second1_6',
 'skew_browser_version_system_op_diff_second1_9',
 'max_op_city_system_op_diff_second1_9',
 'kurt_url_system_op_diff_second1_15',
 'min_browser_system_op_diff_second1_3',
 'skew_department_system_op_diff_second1_12',
 'avg_http_status_code_system_op_diff_second1_15',
 'avg_url_system_op_diff_second1_6',
 'min_ip_type_system_op_diff_second1_15']

In [11]:
feats2 = ['op_month_ip_transform_browser_version_edge_93_cumsum',
 'system_system_op_times_groups_browser_version_edge_93_cumsum',
 'op_times_groups_device_num_transform_op_city_成都_cumsum',
 'op_month_ip_transform_op_city_成都_cumsum',
 'system_op_days_log_system_transform_nHrKgKdJ1Mzt_cumsum',
 'system_op_days_browser_chrome_cumsum',
 'system_system_op_times_groups_op_city_深圳_cumsum',
 'op_days_user_name_browser_version_edge_93_cumsum',
 'op_times_groups_device_num_transform_url_wpsdoc.xxx.com/download_cumsum',
 'system_op_days_ip_type_内网_error_code_cumsum',
 'op_month_user_name_http_status_code_200_error_code_cumsum',
 'system_op_days_os_version_win10_error_code_cumsum',
 'op_times_groups_device_num_transform_department_rd_cumsum',
 'op_days_ip_transform_department_sales_cumsum',
 'op_month_device_num_transform_browser_version_chrome_90_error_code_cumsum',
 'op_month_ip_transform_op_city_深圳_cumsum',
 'op_times_groups_device_num_transform_browser_edge_cumsum',
 'op_month_ip_transform_op_city_北京_cumsum',
 'op_month_device_num_transform_op_city_北京_error_code_cumsum',
 'system_op_days_ip_type_内网_cumsum',
 'op_times_groups_user_name_browser_chrome_cumsum',
 'op_month_user_name_op_city_杭州_error_code_cumsum',
 'op_month_device_num_transform_cumsum',
 'op_days_ip_transform_cumsum',
 'op_days_user_name_ip_type_内网_cumsum',
 'op_month_ip_transform_cumsum',
 'system_op_days_op_city_杭州_cumsum',
 'op_month_user_name_op_city_深圳_error_code_cumsum',
 'op_days_device_num_transform_op_city_成都_cumsum',
 'system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_cumsum',
 'op_days_ip_transform_url_xxx.com/github_cumsum',
 'op_month_device_num_transform_ip_type_内网_error_code_cumsum',
 'op_days_user_name_department_rd_error_code_cumsum',
 'op_days_ip_transform_op_city_北京_cumsum',
 'op_days_device_num_transform_op_city_杭州_cumsum',
 'system_system_op_times_groups_op_city_杭州_cumsum',
 'system_op_days_op_city_北京_cumsum',
 'system_system_op_times_groups_browser_chrome_cumsum',
 'system_system_op_times_groups_http_status_code_200_cumsum',
 'system_op_days_op_city_深圳_cumsum',
 'op_days_user_name_browser_chrome_cumsum',
 'op_month_ip_transform_ip_type_内网_cumsum',
 'system_system_op_times_groups_department_rd_error_code_cumsum',
 'op_days_device_num_transform_op_city_深圳_cumsum',
 'op_days_ip_transform_http_status_code_200_cumsum']

In [12]:
feats3=['ip_transform_2',
 'ip_transform_3',
 'ip_transform_4',
 'ip_transform_5',
 'user_name_1',
 'user_name_3',
 'user_name_4',
 'device_num_transform_1',
 'device_num_transform_2',
 'device_num_transform_3',
 'device_num_transform_5',
 'department_1',
 'department_2',
 'department_3',
 'department_4',
 'department_5',
 'browser_version_2',
 'browser_version_3',
 'browser_version_5',
 'browser_1',
 'browser_3',
 'os_version_1',
 'os_version_2',
 'os_version_3',
 'os_version_4',
 'os_version_5',
 'op_city_3',
 'op_city_4',
 'log_system_transform_1',
 'url_1',
 'url_2',
 'url_4',
 'url_5']

In [23]:
feats = feats1 + feats2 + feats3

In [21]:
feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [25]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 63,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.927171
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.930713
[3]	valid_0's auc: 0.933887
[4]	valid_0's auc: 0.933913
[5]	valid_0's auc: 0.938878
[6]	valid_0's auc: 0.93884
[7]	valid_0's auc: 0.938823
[8]	valid_0's auc: 0.93946
[9]	valid_0's auc: 0.939302
[10]	valid_0's auc: 0.939375
[11]	valid_0's auc: 0.939118
[12]	valid_0's auc: 0.93906
[13]	valid_0's auc: 0.938811
[14]	valid_0's auc: 0.939524
[15]	valid_0's auc: 0.939417
[16]	valid_0's auc: 0.939488
[17]	valid_0's auc: 0.939419
[18]	valid_0's auc: 0.939318
[19]	valid_0's auc: 0.939891
[20]	valid_0's auc: 0.939895
[21]	valid_0's auc: 0.940778
[22]	valid_0's auc: 0.940962
[23]	valid_0's auc: 0.941029
[24]	valid_0's auc: 0.940924
[25]	valid_0's auc: 0.940925
[26]	valid_0's auc: 0.940938
[27]	valid_0's auc: 0.940964
[28]	valid_0's auc: 0.94097
[29]	valid_0's auc: 0.940917
[30]	valid_0's auc: 0.940881
[31]	valid_0's auc: 0.940887
[32]	valid_0's auc: 0.940907
[33]	vali

[7]	valid_0's auc: 0.932282
[8]	valid_0's auc: 0.93206
[9]	valid_0's auc: 0.931639
[10]	valid_0's auc: 0.931717
[11]	valid_0's auc: 0.932896
[12]	valid_0's auc: 0.933345
[13]	valid_0's auc: 0.933148
[14]	valid_0's auc: 0.933395
[15]	valid_0's auc: 0.933406
[16]	valid_0's auc: 0.936229
[17]	valid_0's auc: 0.93624
[18]	valid_0's auc: 0.936333
[19]	valid_0's auc: 0.937038
[20]	valid_0's auc: 0.937023
[21]	valid_0's auc: 0.937081
[22]	valid_0's auc: 0.937144
[23]	valid_0's auc: 0.937092
[24]	valid_0's auc: 0.937223
[25]	valid_0's auc: 0.93712
[26]	valid_0's auc: 0.936984
[27]	valid_0's auc: 0.93716
[28]	valid_0's auc: 0.937173
[29]	valid_0's auc: 0.937236
[30]	valid_0's auc: 0.937257
[31]	valid_0's auc: 0.937243
[32]	valid_0's auc: 0.93715
[33]	valid_0's auc: 0.937408
[34]	valid_0's auc: 0.937479
[35]	valid_0's auc: 0.937452
[36]	valid_0's auc: 0.937533
[37]	valid_0's auc: 0.937568
[38]	valid_0's auc: 0.93768
[39]	valid_0's auc: 0.937792
[40]	valid_0's auc: 0.937809
[41]	valid_0's auc: 0.9

[29]	valid_0's auc: 0.942512
[30]	valid_0's auc: 0.942446
[31]	valid_0's auc: 0.942449
[32]	valid_0's auc: 0.942502
[33]	valid_0's auc: 0.9426
[34]	valid_0's auc: 0.942548
[35]	valid_0's auc: 0.942469
[36]	valid_0's auc: 0.942462
[37]	valid_0's auc: 0.942442
[38]	valid_0's auc: 0.942393
[39]	valid_0's auc: 0.94242
[40]	valid_0's auc: 0.942448
[41]	valid_0's auc: 0.942532
[42]	valid_0's auc: 0.942479
[43]	valid_0's auc: 0.942491
[44]	valid_0's auc: 0.942535
[45]	valid_0's auc: 0.942681
[46]	valid_0's auc: 0.942637
[47]	valid_0's auc: 0.94265
[48]	valid_0's auc: 0.942654
[49]	valid_0's auc: 0.942607
[50]	valid_0's auc: 0.942631
[51]	valid_0's auc: 0.942533
[52]	valid_0's auc: 0.942498
[53]	valid_0's auc: 0.942474
[54]	valid_0's auc: 0.942477
[55]	valid_0's auc: 0.942446
[56]	valid_0's auc: 0.942461
[57]	valid_0's auc: 0.942479
[58]	valid_0's auc: 0.942495
[59]	valid_0's auc: 0.942496
[60]	valid_0's auc: 0.942218
[61]	valid_0's auc: 0.942239
[62]	valid_0's auc: 0.942239
[63]	valid_0's auc

[90]	valid_0's auc: 0.940429
[91]	valid_0's auc: 0.940439
[92]	valid_0's auc: 0.940463
[93]	valid_0's auc: 0.941041
[94]	valid_0's auc: 0.941437
[95]	valid_0's auc: 0.941251
[96]	valid_0's auc: 0.941253
[97]	valid_0's auc: 0.941251
[98]	valid_0's auc: 0.941247
[99]	valid_0's auc: 0.941251
[100]	valid_0's auc: 0.941294
[101]	valid_0's auc: 0.941533
[102]	valid_0's auc: 0.941461
[103]	valid_0's auc: 0.941771
[104]	valid_0's auc: 0.941611
[105]	valid_0's auc: 0.942524
[106]	valid_0's auc: 0.942537
[107]	valid_0's auc: 0.942319
[108]	valid_0's auc: 0.942281
[109]	valid_0's auc: 0.94199
[110]	valid_0's auc: 0.941979
[111]	valid_0's auc: 0.941992
[112]	valid_0's auc: 0.941948
[113]	valid_0's auc: 0.941917
[114]	valid_0's auc: 0.941934
[115]	valid_0's auc: 0.941711
[116]	valid_0's auc: 0.941889
[117]	valid_0's auc: 0.942327
[118]	valid_0's auc: 0.942348
[119]	valid_0's auc: 0.94237
[120]	valid_0's auc: 0.942379
[121]	valid_0's auc: 0.942361
[122]	valid_0's auc: 0.94238
[123]	valid_0's auc: 0.

,name,importance
108,system_system_op_times_groups_http_status_code...,305561.777543
39,avg_ip_transform_system_op_diff_second1_6,129421.767688
56,avg_ip_type_system_op_diff_second1_3,49479.609034
36,kurt_ip_transform_system_op_diff_second1_6,34334.055410
78,op_times_groups_device_num_transform_url_wpsdo...,23368.401316
46,max_ip_transform_system_op_diff_second1_3,21232.448191
22,min_user_name_system_op_diff_second1_6,19173.629188
51,op_datetime_hour_cos,12720.659283
67,avg_http_status_code_system_op_diff_second1_15,9496.213627
0,op_datetime_hour,6020.256076


0.9418232537782785

0.0019216871554673162

In [42]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, KFold

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'gamma': 0.1,
    'max_depth': 8,
    'alpha': 0,
    'lambda': 0,
    'subsample': 0.7,
    'colsample_bytree': 0.5,
    'min_child_weight': 3,
    'eta': 0.01,
    'nthread': -1,
    'seed': 2019,
}

fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = xgb.DMatrix(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = xgb.DMatrix(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        watchlist = [(train, 'train'), (val, 'valid')]
        model = xgb.train(params, train, 20000, watchlist, verbose_eval=20, early_stopping_rounds=200)

        oof[val_idx] += model.predict(xgb.DMatrix(df_train.loc[val_idx, feats]), ntree_limit=model.best_ntree_limit) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(xgb.DMatrix(df_test[feats]), ntree_limit=model.best_ntree_limit)
        importance += np.array(list(model.get_fscore().values())) / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(xgb.DMatrix(df_train.loc[val_idx, feats]), ntree_limit=model.best_ntree_limit)))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[0]	train-auc:0.93886	valid-auc:0.93136
[20]	train-auc:0.95315	valid-auc:0.94107
[40]	train-auc:0.95573	valid-auc:0.94187
[60]	train-auc:0.95594	valid-auc:0.94112
[80]	train-auc:0.95776	valid-auc:0.94259
[100]	train-auc:0.95878	valid-auc:0.94151
[120]	train-auc:0.95979	valid-auc:0.94131
[140]	train-auc:0.96180	valid-auc:0.94084
[160]	train-auc:0.96265	valid-auc:0.94087
[180]	train-auc:0.96356	valid-auc:0.94148
[200]	train-auc:0.96605	valid-auc:0.94167
[220]	train-auc:0.96719	valid-auc:0.94196
[240]	train-auc:0.96870	valid-auc:0.94162
[260]	train-auc:0.97005	valid-auc:0.94145
[279]	train-auc:0.97136	valid-auc:0.94174
----------- 1
[0]	train-auc:0.94114	valid-auc:0.93425
[20]	train-auc:0.95239	valid-auc:0.94110
[40]	train-auc:0.95581	valid-auc:0.94129
[60]	train-auc:0.95542	valid-auc:0.94057
[80]	train-auc:0.95585	valid-auc:0.93979
[100]	train-auc:0.95702	valid-auc:0.93842
[120]	train-auc:0.95809	valid-auc:0.93864
[140]	train-auc:0.95974	valid-auc:0.93854
[160]	train-auc:0.

,name,importance
51,op_datetime_hour_cos,655.0
56,avg_ip_type_system_op_diff_second1_3,626.0
67,avg_http_status_code_system_op_diff_second1_15,527.8
0,op_datetime_hour,486.0
60,avg_os_type_system_op_diff_second1_15,411.4
89,system_op_days_ip_type_内网_cumsum,395.8
35,op_diff_second1,390.8
36,kurt_ip_transform_system_op_diff_second1_6,390.0
115,ip_transform_2,385.4
44,kurt_http_status_code_system_op_diff_second1_9,355.4


0.9426426474805787

0.0020232811687994593